In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get('llama-3.1-GAD')

In [ ]:
import requests
import json

input_file_path = "sample_data/some-100.json"
output_file_path = "sample_data/llama_3.1-GAD_output.json"

# input_file_path = "sample_data/test_original.json"
# output_file_path = "sample_data/test_output.json"

In [ ]:
with open(input_file_path, "r") as input_file:
    input_data = json.load(input_file)

In [ ]:
from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

base_instruction = """ When given the below JSON formatted file content, I need you to give me the specific sentences from the text that exhibit a set of symptoms which are under the annotations section. Below is an example of INPUT and OUTPUT. Just give me the outputs, do not give any code or re-output the prompting, do not just give "yes" or "no" again, give the actual sentences. Keep JSON Formatting. And make sure to always return all the components of each of the objects, so always include "post_text", "post_title", and "annotations":

Input:

{
{
"post_title": "How do I let go of my anxiety over dying?", "post_text": "It is all I can think about for the last 3 years. During this time, I have shut out my family and cut off all of my friends because I don’t see the point in making meaningful connections. I have lost interest in everything I used to enjoy because I’m too scared to leave my house. It’s affecting my quality of life and I’m starting to think of killing myself to feel like I have some control. Has anyone felt like this and if so, have you and how did you overcome it?",
"annotations": [
"Feeling-nervous-anxious-or-on-edge": [ ]
, "Not-being-able-to-stop-or-control-worrying": [ ]
, "Worrying-too-much-about-different-things": [ ]
, "Trouble-relaxing": [ ]
, "Being-so-restless-that-it-is-hard-to-sit-still": []
, "Becoming-easily-annoyed-or-irritable": []
, "Feeling-afraid-as-if-something-awful-might-happen":[ ]
]
}
}

And here is the example Output:
{
    "post_title": "How do I let go of my anxiety over dying?",
    "post_text": "It is all I can think about for the last 3 years. During this time, I have shut out my family and cut off all of my friends because I don’t see the point in making meaningful connections. I have lost interest in everything I used to enjoy because I’m too scared to leave my house. It’s affecting my quality of life and I’m starting to think of killing myself to feel like I have some control. Has anyone felt like this and if so, have you and how did you overcome it?",
    "annotations": {
        "Feeling-nervous-anxious-or-on-edge": [
            "It is all I can think about for the last 3 years.",
            "I’m too scared to leave my house."
        ],
        "Not-being-able-to-stop-or-control-worrying": [
            "It is all I can think about for the last 3 years."
        ],
        "Worrying-too-much-about-different-things": [],
        "Trouble-relaxing": [
            "I’m too scared to leave my house."
        ],
        "Being-so-restless-that-it-is-hard-to-sit-still": [],
        "Becoming-easily-annoyed-or-irritable": [],
        "Feeling-afraid-as-if-something-awful-might-happen": [
            "It’s affecting my quality of life and I’m starting to think of killing myself to feel like I have some control."
        ]
    }
}


May I proceed with the rest of the INPUTS? """

In [ ]:
import json

results = []

# Function to process each JSON object and collect the output
def process_and_collect_output(obj):
    # Construct the prompt
    full_prompt = f"{base_instruction} {json.dumps(obj, indent=2)}"

    # Prepare the payload
    payload = {
        "messages": [
            {"role": "user", "content": full_prompt}
        ],
        "model": "llama-3.1-8b-instant"
    }

    try:
        # Call the Groq API
        chat_completion = client.chat.completions.create(**payload)

        # Extract the LLM output
        llm_output = chat_completion.choices[0].message.content

        # Clean up the LLM output (remove unwanted characters or text if necessary)
        llm_output = llm_output.strip()  # Strip any extra whitespace
        # Handle cases where output is not valid JSON
        try:
            result = json.loads(llm_output)
        except json.JSONDecodeError:
            print(f"Error decoding JSON from output: {llm_output}")
            result = {}  # Use an empty dictionary or handle as needed

        # Append output to results list
        results.append(result)

    except Exception as e:
        print(f"Error processing object: {e}")

# Process and collect results for each object
for obj in input_data:
    process_and_collect_output(obj)

# Write results to output file with comma-separated JSON objects
with open(output_file_path, "w") as output_file:
    output_file.write("[\n")  # Start the JSON array
    first = True
    for result in results:
        if not first:
            output_file.write(",\n")
        json.dump(result, output_file, indent=2)
        first = False
    output_file.write("\n]")  # End the JSON array


Error decoding JSON from output: {
    "post_title": "Stuck between moving forward and killing myself",
    "post_text": "Some background information: My GF of almost 3 years broke up with me over half a year ago and ever since I have been severely depressed...I couldn't do anything but cry and wishing she would come back...I went to a psychologist and he diagnosed me with Major Depressive Disorder (MDD)...I still had friendly contact with my ex, but I was just hoping she would come back. After an emotional call, she told me to take a few months for myself. And so I did, but I just got worse. I couldn't do anything...October of last year, I contacted her again and we met for the first time in months...It was supposed to just be friendly, but talking about the relationship was inevitable. She said to me she would never take me back...It broke me even more. The next day I sent her a goodbye text and was close to killing myself. She called just in time and we had a good talk. Though a few